In [1]:
import numpy as np
import os,sys,inspect
import importlib

randseed = 13344

# import my functions:
dirpath = os.getcwd()
i = 0
while(os.path.basename(dirpath) != "IEMS450_HW") and i <10:
    dirpath = os.path.dirname(dirpath) 
    i = i + 1
targetdir = dirpath + '/HW1/functions'
if targetdir not in sys.path:
    sys.path.insert(0,targetdir)

import objective_function as obj
importlib.reload(obj)

<module 'objective_function' from '/Users/jingyuanwang/GitHub/IEMS450_HW/HW1/functions/objective_function.py'>

# Initialize

In [2]:
# set up the problem
np.random.seed(randseed)
m, L = 0.01, 1
D = 10 ** np.random.rand(100)
D = (D - np.min(D)) / (np.max(D) - np.min(D)) 
A = np.diag(m + D * (L - m))

# save the function
f = obj.f(A)

# optimization

In [3]:
# starting point
x0 = np.random.rand(100)

In [4]:
import opt_quadraticfunc as opt_q
importlib.reload(opt_q)

model1 = opt_q.steepest_descent(f, alpha=1/(2*f.L))
model2 = opt_q.steepest_descent(f)

In [5]:
x1 = model1.run(x0)

iter 0: obj = 11.467056246360487, improve = 9987.53294375364
iter 1: obj = 9.03150945230463, improve = 2.435546794055856
iter 2: obj = 6.950329613231916, improve = 2.081179839072715
iter 3: obj = 5.209909799212674, improve = 1.7404198140192415
iter 4: obj = 3.7926007030612556, improve = 1.4173090961514188
iter 5: obj = 2.675121858905328, improve = 1.1174788441559276
iter 6: obj = 1.8266665678094376, improve = 0.8484552910958905
iter 7: obj = 1.2077788258194597, improve = 0.6188877419899779
iter 8: obj = 0.7730853476708783, improve = 0.4346934781485814
iter 9: obj = 0.4797513614308029, improve = 0.29333398624007545
iter 10: obj = 0.29082508918517935, improve = 0.18892627224562353
iter 11: obj = 0.16942326495883916, improve = 0.12140182422634019
iter 12: obj = 0.08964211828934623, improve = 0.07978114666949293
iter 13: obj = 0.05243921091375262, improve = 0.03720290737559361
iter 14: obj = 0.1389800564211786, improve = -0.086540845507426


In [6]:
x2 = model2.run(x0)

iter 0: obj = 11.467056246360487, improve = 9987.53294375364
iter 1: obj = 5.631684099800234, improve = 5.835372146560253
iter 2: obj = 2.0315729037394084, improve = 3.6001111960608254
iter 3: obj = 0.7559036850603693, improve = 1.275669218679039
iter 4: obj = 0.644408700314889, improve = 0.11149498474548036
iter 5: obj = 0.4252502451320937, improve = 0.21915845518279525
iter 6: obj = 0.48698009505208295, improve = -0.061729849919989255


In [8]:
x1

array([ 6.56261750e-06, -4.47581001e-04,  4.30619199e-03,  1.17832880e-01,
        1.50437687e-01, -4.64044812e-05, -7.89826093e-03,  8.92728549e-02,
       -5.90389512e-03, -1.75275575e-03,  3.99054113e-02, -5.62245945e-03,
        1.05047858e-01, -1.16941540e-02, -8.22575572e-03,  3.50508823e-02,
        3.88727076e-02,  7.12260959e-02,  3.32946008e-03, -2.06230690e-02,
        2.80580380e-04,  1.29498044e-01, -4.47122430e-04,  5.76837128e-01,
       -5.37603632e-03,  2.62209592e-02,  1.01875547e-01,  8.34122186e-03,
        6.57626235e-02, -2.54433414e-04,  3.85593839e-03, -2.34548063e-06,
        1.11205369e-01, -1.58485726e-04,  1.84395036e-01,  1.57036492e-01,
       -4.96446181e-05,  1.71451110e-01,  4.36401264e-03,  2.64803053e-02,
        1.16280947e-01, -3.10469910e-04,  1.73606037e-04, -8.66743717e-06,
        6.06795073e-02,  2.13125241e-03,  2.47854628e-03,  4.63694402e-02,
        9.19170437e-04,  1.68002435e-05,  1.52048832e-01, -2.23138297e-04,
        4.75790896e-01,  

In [7]:
x2

array([-1.83915478e-06,  2.58952312e-02, -1.44074388e-05,  3.79888572e-01,
        3.70493013e-01,  2.40999194e-02,  5.41178966e-03,  2.38393991e-01,
        6.41754934e-03,  8.80111921e-04,  2.18112091e-01,  4.50982267e-03,
        1.26216173e-01,  8.82086845e-02,  8.17198118e-03,  1.71164016e-01,
        5.87113111e-02,  1.10770434e-01,  4.46630215e-05,  1.28964871e-02,
        2.46580136e-03,  2.03239248e-01,  2.56406397e-02,  6.73427680e-01,
        6.17801766e-02,  1.04922902e-01,  1.39200346e-01,  1.42985649e-01,
        1.43764323e-01,  1.45573693e-02, -1.38329844e-04, -4.38956908e-05,
        3.22673674e-01,  1.30927059e-06,  4.85133224e-01,  2.50964922e-01,
        1.59071117e-02,  2.84569723e-01,  5.62222669e-05,  1.52048902e-01,
        3.86518662e-01,  4.60220785e-05, -5.68308642e-05,  2.79383926e-03,
        2.06332605e-01, -1.79568156e-05,  2.64795345e-02,  2.76200644e-01,
       -6.61807609e-05,  9.44746545e-03,  2.75132165e-01,  1.36134443e-04,
        6.93657693e-01,  

## check scipy

In [9]:
import scipy.optimize as opt

In [10]:
opt.minimize(f.value, x0, jac=f.der_1st, hess=f.der_2nd, method = 'Newton-CG')

     fun: 1.533178228775214e-14
     jac: array([-1.81703592e-09, -2.92767500e-08, -2.11677053e-08,  3.68500997e-08,
        5.24844896e-09, -8.01323570e-08, -3.38024929e-08,  3.54369763e-08,
        8.43143798e-08,  5.74929290e-09,  5.29057873e-09,  4.66928826e-08,
       -9.53079295e-10, -1.15089140e-07,  1.80227721e-07,  3.31493082e-08,
        1.41881772e-08,  1.44703172e-09, -1.22220731e-09,  3.84597030e-08,
       -6.16770418e-10,  1.62269619e-08, -2.33685369e-08,  5.69969753e-08,
       -6.48014810e-08, -5.75393912e-08,  2.20405201e-08, -9.87080164e-08,
       -7.20509913e-08, -1.53951969e-08,  2.07005597e-09, -4.16601560e-08,
       -1.94086195e-08, -1.86216390e-08,  1.48127774e-08, -7.16126162e-09,
        6.84115072e-08, -2.98280127e-09, -1.90053303e-08,  2.13020306e-10,
        7.35403484e-08, -1.21323852e-08, -5.58794347e-08, -8.24891417e-09,
        5.43967999e-08,  4.40852844e-08, -1.99042072e-08,  2.20078143e-10,
       -1.49682005e-08,  3.58680645e-09,  1.84021271e-08, 